In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from matplotlib import pyplot
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, classification_report

def preprocess_pandas(data):
    data['Sentence'] = data['Sentence'].str.lower()
    data['Sentence'] = data['Sentence'].replace('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '', regex=True)                      # remove emails
    data['Sentence'] = data['Sentence'].replace('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '', regex=True)    # remove IP address
    data['Sentence'] = data['Sentence'].str.replace('[^\w\s]','')                                                       # remove special characters
    data['Sentence'] = data['Sentence'].replace('\d', '', regex=True)                                                   # remove numbers
    return data

# If this is the primary file that is executed (ie not an import of another file)
if __name__ == "__main__":
    # get data, pre-process and split
    data = pd.read_csv("anaconda3/Exercises/amazon_cells_labelled.txt", delimiter='\t', header=None)
    data.columns = ['Sentence', 'Class']
    data['index'] = data.index                                          # add new column index
    columns = ['index', 'Class', 'Sentence']
    data = preprocess_pandas(data)                             # pre-process
    training_data, validation_data, training_labels, validation_labels = train_test_split( # split the data into training, validation, and test splits
        data['Sentence'].values.astype('U'),
        data['Class'].values.astype('int32'),
        test_size=0.10,
        random_state=0,
        shuffle=True
    )

    # vectorize data using TFIDF and transform for PyTorch for scalability
    word_vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=50000, max_df=0.5, use_idf=True, norm='l2')
    training_data = word_vectorizer.fit_transform(training_data)        # transform texts to sparse matrix
    training_data = training_data.todense()                             # convert to dense matrix for Pytorch
    vocab_size = len(word_vectorizer.vocabulary_)
    validation_data = word_vectorizer.transform(validation_data)
    validation_data = validation_data.todense()
    train_x_tensor = torch.from_numpy(np.array(training_data)).type(torch.FloatTensor)
    train_y_tensor = torch.from_numpy(np.array(training_labels)).long()
    validation_x_tensor = torch.from_numpy(np.array(validation_data)).type(torch.FloatTensor)
    validation_y_tensor = torch.from_numpy(np.array(validation_labels)).long()

In [8]:
# Define the neural network architecture
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

# Initialize the model, optimizer, and loss function
input_size = vocab_size
hidden_size = 128
output_size = 2
model = SimpleNN(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Train the model
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    output = model(train_x_tensor)
    loss = criterion(output, train_y_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item()}")

# Define the chatbot
def chatbot():
    print("Chatbot: Hello! How can I assist you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        else:
            # Preprocess user input and convert to tensor
            user_input = word_vectorizer.transform([user_input])
            user_input = torch.tensor(user_input.todense(), dtype=torch.float32)
            # Analyze user input using the trained model
            model.eval()
            with torch.no_grad():
                output = model(user_input)
                predicted_class = torch.argmax(output, dim=1).item()
            if predicted_class == 0:
                print("Chatbot: It seems like you're expressing a negative sentiment.")
            else:
                print("Chatbot: It seems like you're expressing a positive sentiment.")

# Start the chatbot
chatbot()

Epoch [1/100], Loss: 0.6930196285247803
Epoch [2/100], Loss: 0.6882813572883606
Epoch [3/100], Loss: 0.6833800077438354
Epoch [4/100], Loss: 0.6780531406402588
Epoch [5/100], Loss: 0.6721526384353638
Epoch [6/100], Loss: 0.6655877828598022
Epoch [7/100], Loss: 0.6583189368247986
Epoch [8/100], Loss: 0.6503648161888123
Epoch [9/100], Loss: 0.6417384147644043
Epoch [10/100], Loss: 0.6324566006660461
Epoch [11/100], Loss: 0.6225454807281494
Epoch [12/100], Loss: 0.6120348572731018
Epoch [13/100], Loss: 0.6009528040885925
Epoch [14/100], Loss: 0.5893341302871704
Epoch [15/100], Loss: 0.5772116184234619
Epoch [16/100], Loss: 0.5646151900291443
Epoch [17/100], Loss: 0.5515804886817932
Epoch [18/100], Loss: 0.5381439328193665
Epoch [19/100], Loss: 0.5243374109268188
Epoch [20/100], Loss: 0.5102003812789917
Epoch [21/100], Loss: 0.49577242136001587
Epoch [22/100], Loss: 0.4810952842235565
Epoch [23/100], Loss: 0.46621203422546387
Epoch [24/100], Loss: 0.4511665105819702
Epoch [25/100], Loss: 0

You:  This does not work!


Chatbot: It seems like you're expressing a negative sentiment.


You:  Im so happy with the support i have got from you!


Chatbot: It seems like you're expressing a positive sentiment.


You:  exit


Chatbot: Goodbye!
